In [2]:
from google.colab import files
import zipfile
import os

# Upload ZIP file
uploaded = files.upload()  # Browse and select the ZIP file

# Extract the uploaded zip file (assuming it's named 'animal_data.zip')
with zipfile.ZipFile("facial_expressions.zip", 'r') as zip_ref:
    zip_ref.extractall("facial_expressions")  # Extracts to a folder in current directory

print("Extraction complete!")

Saving facial_expressions.zip to facial_expressions (1).zip
Extraction complete!


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16,ResNet50,MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [2]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)



image_size=(224,224)
train_generator = data_gen.flow_from_directory("facial_expressions/train",  # or correct subfolder
                                               target_size=image_size,
                                               batch_size=32,
                                               class_mode='categorical',
                                               subset='training')

val_generator=data_gen.flow_from_directory("facial_expressions/test",target_size=image_size,
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset="validation")
base_model=MobileNetV2(weights="imagenet",include_top=False,input_shape=image_size+(3,))
base_model.trainable=False

Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [3]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units=7,activation='softmax'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 62720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       8,028,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,287,687 (39.24 MB)

 Trainable params: 8,029,447 (30.63 MB)

 Non-trainable params: 2,258,240 (8.61 MB)

In [4]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_generator, validation_data= val_generator, epochs = 10, verbose = 2)

Epoch 1/10
718/718 - 2051s - 3s/step - accuracy: 0.2938 - loss: 1.8690 - val_accuracy: 0.3596 - val_loss: 1.6329
Epoch 2/10
718/718 - 789s - 1s/step - accuracy: 0.3488 - loss: 1.6478 - val_accuracy: 0.3638 - val_loss: 1.6594
Epoch 3/10
718/718 - 869s - 1s/step - accuracy: 0.3634 - loss: 1.6182 - val_accuracy: 0.3806 - val_loss: 1.5736
Epoch 4/10
718/718 - 904s - 1s/step - accuracy: 0.3666 - loss: 1.6109 - val_accuracy: 0.4008 - val_loss: 1.5789
Epoch 5/10
718/718 - 931s - 1s/step - accuracy: 0.3756 - loss: 1.5998 - val_accuracy: 0.3925 - val_loss: 1.6024
Epoch 6/10
718/718 - 974s - 1s/step - accuracy: 0.3725 - loss: 1.5960 - val_accuracy: 0.4043 - val_loss: 1.5592
Epoch 7/10
718/718 - 958s - 1s/step - accuracy: 0.3832 - loss: 1.5860 - val_accuracy: 0.3778 - val_loss: 1.5834
Epoch 8/10
718/718 - 980s - 1s/step - accuracy: 0.3827 - loss: 1.5818 - val_accuracy: 0.3966 - val_loss: 1.5495
Epoch 9/10
718/718 - 831s - 1s/step - accuracy: 0.3838 - loss: 1.5751 - val_accuracy: 0.3715 - val_loss

In [ ]:
# Set image size and paths
image_size = (224, 224)
batch_size = 32


# ✅ Image Data Generator with Augmentation
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ✅ Data Generators
train_generator = data_gen.flow_from_directory(
    "facial_expressions/train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = data_gen.flow_from_directory(
    "facial_expressions/test",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# ✅ Pre-trained VGG16 model (without top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=image_size + (3,))
base_model.trainable = False  # Freeze base model

# ✅ Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 15 classes
])

# ✅ Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ Summary
model.summary()

# ✅ Early Stopping Callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[early_stop],
    verbose=2
)


Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,423,367 (9.24 MB)

 Trainable params: 165,127 (645.03 KB)

 Non-trainable params: 2,258,240 (8.61 MB)

Epoch 1/30


In [ ]:
# Unfreeze last few layers of VGG16
for layer in base_model.layers[-4:]:  # unfreezing last 4 layers
    layer.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # smaller LR!
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=45,
    initial_epoch=10,
    callbacks=[early_stop],
    verbose=2
)


In [ ]:
train_generator = data_gen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=False
)

val_generator = data_gen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

In [ ]:
def evaluate_model(generator, dataset_type="Validation"):
    # Get ground truth labels
    true_labels = generator.classes

    # Get predictions
    pred_probs = model.predict(generator, verbose=0)
    pred_labels = np.argmax(pred_probs, axis=1)

    # Class names
    class_names = list(generator.class_indices.keys())

    # Accuracy
    acc = accuracy_score(true_labels, pred_labels)
    print(f"\n📊 {dataset_type} Accuracy: {acc:.4f}\n")

    # Classification report
    print(f"📄 {dataset_type} Classification Report:")
    print(classification_report(true_labels, pred_labels, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()


In [ ]:
evaluate_model(train_generator, dataset_type="Training")
evaluate_model(val_generator, dataset_type="Validation")

In [ ]:
import os
os.makedirs('model', exist_ok=True)
model.save('model/facial_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('model/facial_model.h5')
